#Carga de Datos

##Librerias spark

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main

In [2]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

##Datos panda

In [98]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

path = 'drive/MyDrive/TPs-Datos-2025-2C/data'

categories = pd.read_csv(path + '/categories.csv')
customers = pd.read_csv(path + '/customers.csv')
# inventory_logs = pd.read_csv(path + '/inventory_logs.csv')
order_items = pd.read_csv(path + '/order_items.csv')
orders = pd.read_csv(path + '/orders.csv')
products = pd.read_csv(path + '/products.csv')
# reviews = pd.read_csv(path + '/reviews.csv')

##Datos Spark

In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [99]:
sqlContext = SQLContext(sc)
path = 'drive/MyDrive/TPs-Datos-2025-2C/data'

# DataFrames de Spark
categoriesSpark = sqlContext.read.csv(path + '/categories.csv', header=True, inferSchema=True)
customersSpark = sqlContext.read.csv(path + '/customers.csv', header=True, inferSchema=True)
# inventoryLogsSpark = sqlContext.read.csv(path + '/inventory_logs.csv', header=True, inferSchema=True)
orderItemsSpark = sqlContext.read.csv(path + '/order_items.csv', header=True, inferSchema=True)
ordersSpark = sqlContext.read.csv(path + '/orders.csv', header=True, inferSchema=True)
productsSpark = sqlContext.read.csv(path + '/products.csv', header=True, inferSchema=True)
# reviewsSpark = sqlContext.read.csv(path + '/reviews.csv', header=True, inferSchema=True)

# RDDs asociados
categoriesRDD = categoriesSpark.rdd
customersRDD = customersSpark.rdd
# inventoryLogsRDD = inventoryLogsSpark.rdd
orderItemsRDD = orderItemsSpark.rdd
ordersRDD = ordersSpark.rdd
productsRDD = productsSpark.rdd
# reviewsRDD = reviewsSpark.rdd



/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


#Consultas

##1) Cuál es el estado que más descuentos tiene en total? y en promedio? Supongan que de una direccion del estilo: 3123 Alan Extension Port Andrea, MA 26926, “MA” es el estado.

In [ ]:
ordersRDD.take(18)

[Row(_c0=0, order_id=1, customer_id=447917, order_date='2024-07-26T03:04:05.462241', status='completed', payment_method='Digital Wallet', shipping_address='49599 Wesley Burg Richardview, AZ 30649', billing_address='USNV Morrison FPO AP 90901', discount_amount=0.0, tax_amount=None, shipping_cost=None, total_amount=25.96, currency='USD', created_at='2024-07-26T03:04:05.462241', updated_at='2024-07-27T03:04:05.462241', subtotal=0),
 Row(_c0=1, order_id=2, customer_id=441379, order_date='2025-03-22T22:45:14.166543', status='Completed', payment_method='Cash on Delivery', shipping_address='3123 Alan Extension Port Andrea, MA 26926', billing_address='95271 Russell Spurs Apt. 613 West Marcus, ND 52394', discount_amount=0.0, tax_amount=0, shipping_cost=43.36, total_amount=43.36, currency='USD ', created_at='2025-03-22T22:45:14.166543', updated_at=None, subtotal=0),
 Row(_c0=2, order_id=3, customer_id=420585, order_date=None, status='Refunded', payment_method='PayPal', shipping_address='97369 Br

In [ ]:
filtrarEstados = (lambda x: x.shipping_address.split()[-2])

#Me quedo con los estados de la columna shipping_address

fila = ordersRDD.filter(lambda x: (x.customer_id == 471373 ))\
.take(1)[0]
# .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
# .map(lambda x: (filtrarEstados(x),(x.discount_amount,1)))\

# Mostramos tipo de cada atributo
for campo in fila.__fields__:
    valor = getattr(fila, campo)
    print(f"{campo}: {valor} -> {type(valor)}")

_c0: 17 -> <class 'int'>
order_id: 18 -> <class 'int'>
customer_id: 471373 -> <class 'int'>
order_date: 2025-06-14T08:19:15.278706 -> <class 'str'>
status: None -> <class 'NoneType'>
payment_method: Debit Card -> <class 'str'>
shipping_address: undefined -> <class 'str'>
billing_address: 80704 Simmons Knoll Suite 707 Port Jacob, AZ 68989 -> <class 'str'>
discount_amount: 0.0 -> <class 'float'>
tax_amount: 0 -> <class 'int'>
shipping_cost: None -> <class 'NoneType'>
total_amount: 6.21 -> <class 'float'>
currency: EUR -> <class 'str'>
created_at: 2025-06-14T08:19:15.278706 -> <class 'str'>
updated_at: 2025-06-18T08:19:15.278706 -> <class 'str'>
subtotal: 0 -> <class 'int'>


In [ ]:
fila2 = ordersRDD.filter(lambda x: (x.customer_id == 431099 ))\
.take(1)[0]

# Mostramos tipo de cada atributo
for campo in fila.__fields__:
    valor = getattr(fila, campo)
    print(f"{campo}: {valor} -> {type(valor)}")

_c0: 11 -> <class 'int'>
order_id: 12 -> <class 'int'>
customer_id: 431099 -> <class 'int'>
order_date: 2023-10-01T14:15:43.471806 -> <class 'str'>
status: None -> <class 'NoneType'>
payment_method: PayPal  -> <class 'str'>
shipping_address: None -> <class 'NoneType'>
billing_address: 127 Payne Loaf Suite 346 Adammouth, NV 52210 -> <class 'str'>
discount_amount: 0.0 -> <class 'float'>
tax_amount: 0 -> <class 'int'>
shipping_cost: 21.32 -> <class 'float'>
total_amount: None -> <class 'NoneType'>
currency: EUR -> <class 'str'>
created_at: 2023-10-01T14:15:43.471806 -> <class 'str'>
updated_at: 2023-10-03T14:15:43.471806 -> <class 'str'>
subtotal: 0 -> <class 'int'>


In [ ]:
filtrarEstados = (lambda x: x.shipping_address.split()[-2])

#Me quedo con los estados de la columna shipping_address

ordersRDD.filter(lambda row: row.shipping_address is not None and row.shipping_address.lower() != 'undefined' and row.shipping_address.strip() != '')\
.take(17)
# .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
# .map(lambda x: (filtrarEstados(x),(x.discount_amount,1)))\

[Row(_c0=0, order_id=1, customer_id=447917, order_date='2024-07-26T03:04:05.462241', status='completed', payment_method='Digital Wallet', shipping_address='49599 Wesley Burg Richardview, AZ 30649', billing_address='USNV Morrison FPO AP 90901', discount_amount=0.0, tax_amount=None, shipping_cost=None, total_amount=25.96, currency='USD', created_at='2024-07-26T03:04:05.462241', updated_at='2024-07-27T03:04:05.462241', subtotal=0),
 Row(_c0=1, order_id=2, customer_id=441379, order_date='2025-03-22T22:45:14.166543', status='Completed', payment_method='Cash on Delivery', shipping_address='3123 Alan Extension Port Andrea, MA 26926', billing_address='95271 Russell Spurs Apt. 613 West Marcus, ND 52394', discount_amount=0.0, tax_amount=0, shipping_cost=43.36, total_amount=43.36, currency='USD ', created_at='2025-03-22T22:45:14.166543', updated_at=None, subtotal=0),
 Row(_c0=2, order_id=3, customer_id=420585, order_date=None, status='Refunded', payment_method='PayPal', shipping_address='97369 Br

In [ ]:
def tieneDireccionValida(orden):
    return (
        orden.shipping_address is not None
        and orden.shipping_address.lower() != "undefined"
        and orden.shipping_address.strip() != ""
    )

def obtenerEstado(orden):
    return orden.shipping_address.split()[-2]

def llenarDescuento(orden):
    return orden.discount_amount if orden.discount_amount is not None else 0

def combinarDescuentos(x, y):
    sumaDescuentos = x[0] + y[0]
    cantTotalOrdenes = x[1] + y[1]
    return ( sumaDescuentos, cantTotalOrdenes )

def promedioPorDescuento( totalDescuentos , cantTotalOrdenes ):
    return totalDescuentos / cantTotalOrdenes

def mayorPorTotal(x, y):
    return x if x[1][0] > y[1][0] else y

#Pipeline
acumuladoPorEstado = ordersRDD.filter(tieneDireccionValida)\
    .map(lambda orden: ( obtenerEstado(orden), (llenarDescuento(orden),1) ))\
    .reduceByKey( combinarDescuentos )

estadisticasPorEstado = acumuladoPorEstado.mapValues( lambda x: (x[0], promedioPorDescuento(x[0],x[1])) )

estadisticasPorEstado.reduce( mayorPorTotal )

('AP', (1550705.2400000007, 10.670602029932914))

##2) ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado ‘Refunded’? ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

In [42]:
def tieneDireccionValida(orden):
    return (
        orden.shipping_address is not None
        and orden.shipping_address.lower() != "undefined"
        and orden.shipping_address.strip() != ""
    )

def obtenerCodigoPostal(orden):
    return orden.shipping_address.split()[-1]

def combinarOrdenesRefunded(x, y):
    cantTotalOrdenes = x + y
    return cantTotalOrdenes

#Pipeline

# 5 codigos mas comunes de Refunded
refundedOrders = ordersRDD.filter( lambda x: x.status== 'Refunded' ).filter( tieneDireccionValida ).cache()
# topZipCodes
refundedOrders.map(lambda orden: ( obtenerCodigoPostal(orden), 1 ) ).reduceByKey( combinarOrdenesRefunded ).takeOrdered( 5, key=lambda x: -x[1] )
  # [('70696', 6), ('47612', 5), ('99202', 5), ('75585', 4), ('52309', 4)]

[('70696', 6), ('47612', 5), ('99202', 5), ('75585', 4), ('52309', 4)]

In [43]:
filtrarZipCode = lambda x: (
      obtenerCodigoPostal(x) == '70696'
      or obtenerCodigoPostal(x) == '47612'
      or obtenerCodigoPostal(x) == '99202'
      or obtenerCodigoPostal(x) == '75585'
      or obtenerCodigoPostal(x) == '52309'
)

ClientesZipCodes = refundedOrders.filter( filtrarZipCode ).map( lambda x: ( x.customer_id, ( obtenerCodigoPostal(x) , 1 ) ) ).cache()
ClientesZipCodes.count()

24

In [44]:
#Verificando si se repiten
ClientesZipCodes.reduceByKey( lambda x,y: x[1][1]+y[1][1] ).take(24)

[(438664, ('75585', 1)),
 (483592, ('52309', 1)),
 (446944, ('47612', 1)),
 (458648, ('47612', 1)),
 (475792, ('47612', 1)),
 (431809, ('75585', 1)),
 (415305, ('70696', 1)),
 (435545, ('52309', 1)),
 (422050, ('52309', 1)),
 (436994, ('70696', 1)),
 (471187, ('99202', 1)),
 (478707, ('70696', 1)),
 (406955, ('99202', 1)),
 (494827, ('99202', 1)),
 (498627, ('70696', 1)),
 (485828, ('75585', 1)),
 (424157, ('70696', 1)),
 (452549, ('99202', 1)),
 (444925, ('47612', 1)),
 (485534, ('75585', 1)),
 (405566, ('70696', 1)),
 (485775, ('47612', 1)),
 (437671, ('99202', 1)),
 (450183, ('52309', 1))]

In [46]:
clientesPorId = customersRDD.map( lambda x: ( x.customer_id, (x.first_name , x.last_name )) )
clientesNombreApellido = ClientesZipCodes.join( clientesPorId ).cache()
clientesNombreApellido.count()

24

In [47]:
clientesNombreApellido.take(24)

[(422050, (('52309', 1), ('Briana', 'DANIELS'))),
 (437671, (('99202', 1), ('Shawn', 'Graham'))),
 (483592, (('52309', 1), ('Jennifer', 'Martin'))),
 (475792, (('47612', 1), ('Patrick', 'Maxwell'))),
 (450183, (('52309', 1), ('Dana', 'Summers'))),
 (438664, (('75585', 1), ('Emily', 'Garner'))),
 (485534, (('75585', 1), ('Melinda', 'MILLER '))),
 (446944, (('47612', 1), ('Jodi', 'Carter'))),
 (436994, (('70696', 1), ('Amber', 'Campbell'))),
 (485775, (('47612', 1), ('Donna', 'Jacobs'))),
 (406955, (('99202', 1), ('Brandon', 'Harrison'))),
 (415305, (('70696', 1), ('Edward', 'Gentry'))),
 (435545, (('52309', 1), ('Ricky', 'Dunn'))),
 (444925, (('47612', 1), ('Debbie', 'Smith'))),
 (405566, (('70696', 1), ('Ashley', 'undefined'))),
 (471187, (('99202', 1), ('WILLIAM', 'Smith'))),
 (478707, (('70696', 1), ('LEE', 'Clark'))),
 (424157, (('70696', 1), ('Chelsea ', None))),
 (494827, (('99202', 1), ('Regina', 'WILLIAMSON'))),
 (498627, (('70696', 1), ('Todd', 'Owen'))),
 (485828, (('75585', 1

##3) Para cada tipo de pago y segmento de cliente, devolver la suma y el promedio expresado como porcentaje, de clientes activos y de consentimiento de marketing. Se valora que el output de la consulta tenga nombres claros y en español.

### cantidad de palabras diff

In [50]:
ordersRDD.map(lambda x: x.payment_method).distinct().count()

56

In [52]:
ordersRDD.map(lambda x: x.payment_method).distinct().collect()

['PayPal',
 None,
 'DIGITAL WALLET ',
 ' CASH ON DELIVERY',
 'bank transfer ',
 ' credit card',
 'PAYPAL ',
 'DEBIT CARD',
 'Credit Card ',
 ' Cash on Delivery',
 'credit card',
 ' Credit Card',
 'Digital Wallet ',
 ' cash on delivery',
 ' debit card',
 'PayPal ',
 'Bank Transfer',
 'CASH ON DELIVERY',
 'Debit Card ',
 'CREDIT CARD ',
 'BANK TRANSFER ',
 'CASH ON DELIVERY ',
 ' BANK TRANSFER',
 'Digital Wallet',
 ' PayPal',
 'Credit Card',
 'PAYPAL',
 ' PAYPAL',
 'digital wallet',
 'cash on delivery ',
 ' digital wallet',
 'digital wallet ',
 'DEBIT CARD ',
 'bank transfer',
 'BANK TRANSFER',
 ' Debit Card',
 ' DIGITAL WALLET',
 ' paypal',
 'paypal',
 'Bank Transfer ',
 'undefined',
 'Cash on Delivery ',
 ' Bank Transfer',
 'debit card',
 'paypal ',
 ' CREDIT CARD',
 'Debit Card',
 'cash on delivery',
 'DIGITAL WALLET',
 'debit card ',
 'Cash on Delivery',
 'CREDIT CARD',
 ' bank transfer',
 ' Digital Wallet',
 'credit card ',
 ' DEBIT CARD']

In [55]:
customersRDD.map(lambda x: x.customer_segment).distinct().count()

29

In [56]:
customersRDD.map(lambda x: x.customer_segment).distinct().collect()

['Regular',
 'Budget',
 'Premium',
 None,
 'regular',
 'premium',
 'Premium ',
 'premium ',
 'PREMIUM',
 'Budget ',
 ' Budget',
 'regular ',
 'REGULAR ',
 ' BUDGET',
 ' budget',
 'budget',
 'undefined',
 'REGULAR',
 'Regular ',
 ' Premium',
 ' Regular',
 'BUDGET',
 'budget ',
 ' regular',
 ' REGULAR',
 ' PREMIUM',
 'BUDGET ',
 ' premium',
 'PREMIUM ']

###resolucion

In [68]:
def normalizarMetodoPago(metodo):
    if metodo is None or metodo.lower().strip() == "undefined":
        return None
    return metodo.lower().strip()

def normalizarSegmento(segmento):
    if segmento is None or segmento.lower().strip() == "undefined":
        return None
    return segmento.lower().strip()


# (customer_id, payment_method)
metodosPorCliente = ordersRDD.map(lambda orden: (
    orden.customer_id,
    normalizarMetodoPago(orden.payment_method)
)).filter(lambda x: x[1] is not None).distinct().cache()

# (customer_id, (segmento, is_active, marketing_consent))
clientesInfo = customersRDD.map(lambda c: (
    c.customer_id,
    (normalizarSegmento(c.customer_segment), c.is_active, c.marketing_consent)
)).filter(lambda x: x[1][0] is not None).cache()

# Join entre ambos
datosCombinados = metodosPorCliente.join(clientesInfo).cache()
# (customer_id, (payment_method, (segmento, is_active, marketing_consent)))
datosCombinados.count()

544587

In [70]:
datosCombinados.take(10)

[(412350, ('bank transfer', ('budget', True, False))),
 (412350, ('digital wallet', ('budget', True, False))),
 (412350, ('credit card', ('budget', True, False))),
 (412350, ('cash on delivery', ('budget', True, False))),
 (412350, ('debit card', ('budget', True, False))),
 (412350, ('paypal', ('budget', True, False))),
 (443950, ('bank transfer', ('premium', True, True))),
 (443950, ('digital wallet', ('premium', True, True))),
 (443950, ('credit card', ('premium', True, True))),
 (443950, ('cash on delivery', ('premium', True, True)))]

In [82]:
# ( (metodo_pago, segmento), (activos, total, consentimiento, total) )
datosAgrupados = datosCombinados.map(lambda x: (
    (x[1][0], x[1][1][0]),  # (payment_method, segmento)
    (1 if x[1][1][1] else 0, 1, 1 if x[1][1][2] else 0, 1)
)).cache()

def combinar(x, y):
    return (x[0]+y[0], x[1]+y[1], x[2]+y[2], x[3]+y[3])

totales = datosAgrupados.reduceByKey(combinar).cache()

resultadoFinal = totales.map(lambda x: (
    x[0][0],   # metodo_pago
    x[0][1],   # segmento
    x[1][0],   # cantidad_clientes_activos
    (x[1][0] / x[1][1]) * 100,  # porcentaje_clientes_activos
    x[1][2],   # cantidad_consentimiento_marketing
    (x[1][2] / x[1][3]) * 100   # porcentaje_consentimiento_marketing
))

# Encabezado con los nombres de las columnas
print(
    "Metodo De Pago | Segmento | Cant Clientes Activos | Porcentaje Clientes Activos | "
    "Cant Consentimiento Marketing | Porcentaje Consentimiento Marketing"
)

resultadoOrdenado = resultadoFinal.sortBy(lambda x: (x[0], x[1]))
for fila in resultadoOrdenado.take(20):
    print(fila)

Metodo De Pago | Segmento | Cant Clientes Activos | Porcentaje Clientes Activos | Cant Consentimiento Marketing | Porcentaje Consentimiento Marketing
('bank transfer', 'budget', 16041, 89.78004141713774, 12451, 69.68713270274807)
('bank transfer', 'premium', 16441, 89.88082221736278, 12808, 70.01968073474744)
('bank transfer', 'regular', 49126, 89.96447276856023, 38297, 70.13331868292862)
('cash on delivery', 'budget', 16040, 89.79454738845659, 12449, 69.69154117449476)
('cash on delivery', 'premium', 16437, 89.87860892388451, 12807, 70.02952755905511)
('cash on delivery', 'regular', 49140, 89.96704503844745, 38307, 70.13365067740754)
('credit card', 'budget', 16039, 89.77889728519452, 12448, 69.67814161768821)
('credit card', 'premium', 16434, 89.87203324948048, 12807, 70.03718691895439)
('credit card', 'regular', 49132, 89.96228073387776, 38301, 70.13036950232541)
('debit card', 'budget', 16034, 89.77603583426652, 12441, 69.65845464725643)
('debit card', 'premium', 16442, 89.88628908

##4) Para los productos que contienen en su descripción la palabra “stuff” (sin importar mayúsculas o minúsculas), calcular el peso total de su inventario agrupado por marca, mostrar sólo la marca y el peso total de las 5 más pesadas.

In [136]:
# Filtrar productos que contengan 'stuff' en la descripción (sin nulos, case insensitive)
productosStuffRDD = productsRDD.filter(
    lambda producto: producto.description is not None and "stuff" in producto.description.lower()
)

# Filtrar marcas válidas (sin undefined o None)
productosFiltradosRDD = productosStuffRDD.filter(
    lambda producto: producto.brand is not None and producto.brand.lower() != "undefined"
)

# Reemplazar nulos con 0 y calcular peso total por producto
productosConPesoRDD = productosFiltradosRDD.map(
    lambda producto: (producto.brand, (producto.stock_quantity or 0) * (producto.weight_kg or 0))
)

# Agrupar por marca y sumar pesos usando lambda
pesoTotalPorMarcaRDD = productosConPesoRDD.reduceByKey(lambda x, y: x + y)


pesoTotalPorMarcaRDD.takeOrdered(5, key=lambda x: -x[1])

# [('3M', 1650353.9800000002),
#  ('Adidas', 1437972.1700000004),
#  ('Wayfair', 1300550.02),
#  ('Hasbro', 1215804.37),
#  ('IKEA', 1121565.1)]

[('3M', 1650353.9800000002),
 ('Adidas', 1437972.1700000004),
 ('Wayfair', 1300550.02),
 ('Hasbro', 1215804.37),
 ('IKEA', 1121565.1)]

##5) Calculen el porcentaje de productos cuyo stock es al menos 20% más alto que el stock promedio de su marca. Por ejemplo, si el stock promedio de la marca Adidas fuera 100, para los productos de dicha marca la condición será que tengan un stock mayor a 120, y luego se deberá calcular qué porcentaje del total de productos cumple con esta condición.

In [122]:
# Filtrar productos válidos y cachear
productosLimpiosRDD = productsRDD.filter(
    lambda x: x.brand is not None and x.brand.lower() != "undefined"
).cache()

# Sumar stock y cantidad de productos por marca
stockPorMarcaRDD = productosLimpiosRDD.map(
    lambda x: (x.brand, (x.stock_quantity or 0, 1))
).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))

# Calcular promedio de stock por marca
promedioStockPorMarcaRDD = stockPorMarcaRDD.map(
    lambda x: (x[0], x[1][0] / x[1][1])
)

# Broadcast del promedio por marca para usar en filter
promedioStockPorMarcaDict = sc.broadcast(dict(promedioStockPorMarcaRDD.collect()))

# Función para verificar si el stock es ≥ 120% del promedio de su marca
def stockMayor20Porc(producto):
    marca = producto.brand
    stock = producto.stock_quantity or 0
    promedio = promedioStockPorMarcaDict.value.get(marca, 0)
    return stock >= 1.2 * promedio

# Filtrar productos que cumplen la condición
productosMayor20PorcRDD = productosLimpiosRDD.filter(stockMayor20Porc)

# Calcular totales y porcentaje
totalProductos = productosLimpiosRDD.count()
productosCumplen = productosMayor20PorcRDD.count()

porcentajeProductosMayor20Porc = (productosCumplen / totalProductos) * 100
porcentajeProductosMayor20Porc

# 41.60567897191177

41.60567897191177

##6) Obtener la cantidad de órdenes que no hayan comprado ninguno de los 10 productos más vendidos.

In [132]:
# Contar cuántas veces se vendió cada producto
cantidadVentasPorProductoRDD = orderItemsRDD.map(lambda x: (x.product_id, 1)) \
    .reduceByKey(lambda x, y: x + y)

# Obtener los 10 productos más vendidos
top10Productos = cantidadVentasPorProductoRDD.takeOrdered(10, key=lambda x: -x[1])
top10IdsProductos = [producto[0] for producto in top10Productos]

# Mapear cada item a (idOrden, flag) según si es top10
ordenesConFlagRDD = orderItemsRDD.map(
    lambda x: (x.order_id, 1 if x.product_id in top10IdsProductos else 0)
)

# Reducir por idOrden para ver si contiene algún top10
def combinarFlags(x, y):
    return 1 if x == 1 or y == 1 else 0

ordenesConTop10FlagRDD = ordenesConFlagRDD.reduceByKey(combinarFlags)

# Filtrar órdenes que NO contienen ninguno de los productos top10
ordenesSinTop10RDD = ordenesConTop10FlagRDD.filter(lambda x: x[1] == 0)

# Contar cantidad de órdenes que cumplen la condición
cantidadOrdenesSinTop10 = ordenesSinTop10RDD.count()
totalOrdenes = orderItemsRDD.map(lambda x: x.order_id).count()

# Calcular porcentaje
porcentajeOrdenesSinTop10 = (cantidadOrdenesSinTop10 / totalOrdenes) * 100

# Mostrar resultados
print("Cantidad de órdenes sin ningún producto del top10:", cantidadOrdenesSinTop10, "de", totalOrdenes, "órdenes totales")

Cantidad de órdenes sin ningún producto del top10: 99490 de 285243 órdenes totales


In [133]:
print("Porcentaje de ordenes sin top10:", cantidadOrdenesSinTop10 / totalOrdenes ,'%')

Porcentaje de ordenes sin top10: 0.3487903296487556 %


##7) Calcular la cantidad y el porcentaje de clientes inactivos por segmento de clientes

In [119]:
# Verificar clientes válidos
def clienteValido(cliente):
    return cliente.customer_segment is not None and cliente.customer_segment.lower().strip() != "undefined"

# Obtener segmento de cliente normalizado
def obtenerSegmento(cliente):
    return cliente.customer_segment.strip().lower()  # eliminar espacios y pasar a minúsculas

# Marcar si el cliente está inactivo
def clienteInactivo(cliente):
    return 1 if not cliente.is_active else 0

# Combinar totales de clientes inactivos y total de clientes por segmento
def combinarInactivos(x, y):
    return (x[0] + y[0], x[1] + y[1])

# Pipeline
acumuladoPorSegmento = customersRDD.filter(clienteValido)\
    .map(lambda c: (obtenerSegmento(c), (clienteInactivo(c), 1)))\
    .reduceByKey(combinarInactivos)

# Calcular porcentaje de inactivos por segmento
estadisticasPorSegmento = acumuladoPorSegmento.mapValues(lambda x: (x[0], x[0]/x[1]))

# Top 5 segmentos con más clientes inactivos
estadisticasPorSegmento.takeOrdered(5, key=lambda x: -x[1][0])

[('regular', (27349, 0.10013583822436374)),
 ('premium', (9245, 0.10116650617175874)),
 ('budget', (8979, 0.09926921758742302))]

## 8) Para cada segmento de cliente, calcular la cantidad total de clientes activos y el porcentaje que representan sobre el total de clientes de ese segmento. Finalmente, devolver el segmento con el mayor porcentaje de clientes activos.

In [110]:
def mapearActivo(cliente):
    return (cliente.customer_segment, (1 if cliente.is_active else 0, 1))

def combinarActivos(x, y):
    return (x[0] + y[0], x[1] + y[1])

def porcentajeActivos(activos, total):
    return (activos / total) if total > 0 else 0

def mayorPorcentaje(x, y):
    return x if x[1][1] > y[1][1] else y

# Pipeline
activosPorSegmento = customersRDD.map(mapearActivo)\
    .reduceByKey(combinarActivos)

estadisticasActivos = activosPorSegmento.mapValues(lambda x: (x[0], porcentajeActivos(x[0], x[1])))

estadisticasActivos.reduce(mayorPorcentaje)

(' BUDGET', (136, 0.9315068493150684))